In [ ]:
# # Análise de Inadimplência em Cartão de Crédito
# ## 1. Informações Básicas
# 
# • Nome do dataset: Default of Credit Card Clients  
# • Fonte: [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)  
# • Dimensões: 30.000 linhas × 25 colunas  
# • Tamanho em MB: ~3.7 MB  

# %%
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Configuração de estilo
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# %%
# Carregar os dados
df = pd.read_excel('../data/default_of_credit_card_clients.xls', header=1)

# Verificar dimensões iniciais
print("=== INFORMAÇÕES INICIAIS ===")
print(f"Dimensões do dataset: {df.shape}")
print(f"Tamanho aproximado em MB: {round(df.memory_usage(deep=True).sum() / (1024**2), 2)}")

# %%
# Visualizar as primeiras linhas
print("\nPrimeiras 5 linhas do dataset:")
df.head()

# %%
# Informações sobre colunas e tipos de dados
print("\nInformações sobre colunas e tipos de dados:")
df.info()

# ## 🎯 VALIDAÇÃO DAS INFORMAÇÕES

# %%
# =============================================================================
# VALIDAÇÃO COMPLETA DOS DADOS
# =============================================================================

print("=== VALIDAÇÃO DOS DADOS ===\n")

# 1. Dimensões confirmadas
print(f"1. Dimensões confirmadas: {df.shape}")

# 2. Tamanho em MB
size_mb = round(df.memory_usage(deep=True).sum() / (1024**2), 2)
print(f"2. Tamanho em MB: {size_mb} MB")

# 3. Renomear coluna target
df.rename(columns={'default payment next month': 'inadimplente'}, inplace=True)
print("3. Coluna target renomeada para 'inadimplente'")

# 4. Distribuição do target
target_dist = df['inadimplente'].value_counts()
print(f"4. Distribuição do target:")
print(f"   - Classe 0 (Não inadimplente): {target_dist[0]} ({target_dist[0]/len(df)*100:.2f}%)")
print(f"   - Classe 1 (Inadimplente): {target_dist[1]} ({target_dist[1]/len(df)*100:.2f}%)")

# 5. Valores nulos
null_count = df.isnull().sum().sum()
print(f"5. Valores nulos totais: {null_count}")
print(f"   - Dataset completo: {null_count == 0}")

# 6. Tipos de dados
print("6. Tipos de dados presentes:")
print(df.dtypes.value_counts())

# 7. Duplicatas
duplicatas = df.duplicated().sum()
print(f"7. Linhas duplicadas: {duplicatas}")

# 8. Estatísticas básicas
print("\n8. Estatísticas descritivas básicas:")
print(df.describe())

# ## 2. Problema de Machine Learning
# 
# • Tipo: [X] Classificação [ ] Regressão  
# • Variável alvo (target): `inadimplente`  
# • Número de classes: 2 (0 = Não inadimplente, 1 = Inadimplente)  
# • Distribuição do target:  
#   - Classe 0: 23.364 casos (77,88%)  
#   - Classe 1: 6.636 casos (22,12%)  
#   - Dataset apresenta desbalanceamento moderado

# ## 3. Qualidade dos Dados
# 
# • % valores faltantes: 0% (dataset completo)  
# • Tipos de variáveis: 
#   - Numéricas contínuas: LIMIT_BAL, BILL_AMT1 até BILL_AMT6, PAY_AMT1 até PAY_AMT6  
#   - Categóricas: SEX, EDUCATION, MARRIAGE, PAY_0 até PAY_6  
#   - Datas: Nenhuma  
#   - Texto: Nenhuma  
# • Qualidade geral: Excelente - dados completos e estruturados

# ## 4. Justificativa
# 
# • Por que escolhemos este dataset? 
#   Dataset clássico e bem-estruturado para aprendizado de classificação binária, com aplicação prática real no setor financeiro. Ideal para aprendizado dos fundamentos de ML.
# 
# • Qual problema real queremos resolver?
#   Prever a inadimplência de clientes com antecedência, permitindo que instituições financeiras reduzam perdas, otimizem a gestão de risco de crédito e criem estratégias de retenção proativas.
# 
# • Que insights esperamos encontrar?
#   Identificar os principais fatores preditivos de inadimplência (histórico de pagamentos, limite de crédito, demografia), entender padrões de comportamento e construir um modelo com boa capacidade preditiva.

# %%
# =============================================================================
# ANÁLISE EXPLORATÓRIA DE DADOS (EDA)
# =============================================================================

# ## 5. Análise Exploratória (EDA)

# %%
# Distribuição da variável target
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
sns.countplot(x='inadimplente', data=df)
plt.title('Distribuição de Inadimplentes')
plt.xlabel('Inadimplente (0 = Não, 1 = Sim)')
plt.ylabel('Contagem')

plt.subplot(1, 2, 2)
plt.pie(target_dist.values, labels=['Não Inadimplente', 'Inadimplente'], 
        autopct='%1.1f%%', colors=['lightblue', 'lightcoral'])
plt.title('Proporção de Inadimplência')
plt.show()

# %%
# Análise de correlações (apenas com algumas variáveis para performance)
print("Correlação com a variável target (inadimplente):")
correlation_with_target = df.corr()['inadimplente'].sort_values(ascending=False)
print(correlation_with_target.head(10))

# Heatmap de correlação das variáveis mais relevantes
plt.figure(figsize=(12, 8))
top_features = correlation_with_target.abs().sort_values(ascending=False).head(10).index
sns.heatmap(df[top_features].corr(), annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Matriz de Correlação - Top 10 Features')
plt.show()

# %%
# Análise de algumas variáveis importantes
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# LIMIT_BAL por inadimplência
sns.boxplot(x='inadimplente', y='LIMIT_BAL', data=df, ax=axes[0,0])
axes[0,0].set_title('Limite de Crédito vs Inadimplência')
axes[0,0].set_xlabel('Inadimplente')
axes[0,0].set_ylabel('Limite de Crédito')

# SEX (Gênero) por inadimplência
sex_counts = pd.crosstab(df['SEX'], df['inadimplente'])
sex_counts.plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Inadimplência por Gênero')
axes[0,1].set_xlabel('Gênero (1=M, 2=F)')
axes[0,1].set_ylabel('Contagem')

# EDUCATION por inadimplência
edu_counts = pd.crosstab(df['EDUCATION'], df['inadimplente'])
edu_counts.plot(kind='bar', ax=axes[1,0])
axes[1,0].set_title('Inadimplência por Educação')
axes[1,0].set_xlabel('Nível Educacional')
axes[1,0].set_ylabel('Contagem')

# AGE por inadimplência
sns.histplot(data=df, x='AGE', hue='inadimplente', bins=30, ax=axes[1,1])
axes[1,1].set_title('Distribuição de Idade por Inadimplência')
axes[1,1].set_xlabel('Idade')

plt.tight_layout()
plt.show()

# ## 6. Preparação para Machine Learning

# %%
# Separar features e target
X = df.drop('inadimplente', axis=1)
y = df['inadimplente']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Shape do X_train: {X_train.shape}")
print(f"Shape do X_test: {X_test.shape}")
print(f"Proporção no treino: {y_train.value_counts(normalize=True).values}")
print(f"Proporção no teste: {y_test.value_counts(normalize=True).values}")

# %%
# Modelo 1: Random Forest
print("=== RANDOM FOREST ===")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_rf))
print(f"Acurácia: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]):.4f}")

# %%
# Modelo 2: Regressão Logística
print("=== REGRESSÃO LOGÍSTICA ===")
lr_model = LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_lr))
print(f"Acurácia: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, lr_model.predict_proba(X_test)[:, 1]):.4f}")

# %%
# Matriz de Confusão - Random Forest
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusão - Random Forest')
plt.ylabel('Verdadeiro')
plt.xlabel('Previsto')

plt.subplot(1, 2, 2)
cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Reds')
plt.title('Matriz de Confusão - Regressão Logística')
plt.ylabel('Verdadeiro')
plt.xlabel('Previsto')

plt.tight_layout()
plt.show()

# %%
# Importância das features - Random Forest
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importance.head(15))
plt.title('Top 15 Features Mais Importantes - Random Forest')
plt.show()

print("Top 10 features mais importantes:")
print(feature_importance.head(10))

# ## 7. Conclusões e Próximos Passos
# 
# Principais Insights:
# - Dataset bem balanceado para um problema de crédito (22% inadimplência)
# - Variáveis de histórico de pagamento (PAY_0 a PAY_6) são as mais importantes
# - Random Forest performou melhor que Regressão Logística
# - Modelo conseguiu capturar padrões preditivos relevantes
# 
# Próximos Passos Sugeridos:
# 1. Tratamento de desbalanceamento (SMOTE, undersampling)
# 2. Engenharia de features mais avançada
# 3. Otimização de hiperparâmetros
# 4. Teste com outros modelos (XGBoost, LightGBM)
# 5. Validação cruzada mais robusta

# %%
# Salvar resultados finais
results = {
    'modelo': ['Random Forest', 'Regressão Logística'],
    'acuracia': [
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_lr)
    ],
    'roc_auc': [
        roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]),
        roc_auc_score(y_test, lr_model.predict_proba(X_test)[:, 1])
    ]
}

results_df = pd.DataFrame(results)
print("=== RESULTADOS FINAIS ===")
print(results_df)